In [1]:
from azureml.core import Workspace, Experiment

ws = Workspace.get(name="quick-starts-ws-123460")
exp = Experiment(workspace=ws, name="udacity-project")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

Performing interactive authentication. Please follow the instructions on the terminal.
To sign in, use a web browser to open the page https://microsoft.com/devicelogin and enter the code FQTAKSFU7 to authenticate.
You have logged in. Now let us find all the subscriptions to which you have access...
Interactive authentication successfully completed.
Workspace name: quick-starts-ws-123460
Azure region: southcentralus
Subscription id: c1dcf523-8ab0-429f-8dba-143a6c49ec84
Resource group: aml-quickstarts-123460


In [2]:
from azureml.core.compute import ComputeTarget, AmlCompute

# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

vm_size = "Standard_D2_V2"
name = "udacity-project"
config = AmlCompute.provisioning_configuration(vm_size=vm_size, max_nodes=4)
compute_cluster = ComputeTarget.create(ws, name, config)
compute_cluster.wait_for_completion(True)

Creating
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


In [16]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform, normal, choice
import os

# Specify parameter sampler
ps = RandomParameterSampling({"C":uniform(0.1,1), "max_iter":choice(20,40,80)})

# Specify a Policy
policy = BanditPolicy(slack_factor=0.1, evaluation_interval=1, delay_evaluation=5)

if "training" not in os.listdir():
    os.mkdir("./training")

# Create a SKLearn estimator for use with train.py
est = SKLearn(source_directory = ".", compute_target= compute_cluster, vm_size= vm_size,entry_script= "train.py")

# Create a HyperDriveConfig using the estimator, hyperparameter sampler, and policy.
hyperdrive_config = HyperDriveConfig(hyperparameter_sampling= ps, primary_metric_name="Accuracy", primary_metric_goal= PrimaryMetricGoal.MAXIMIZE, max_total_runs =100, max_concurrent_runs= 4, policy= policy, estimator= est)

In [17]:
# Submit your hyperdrive run to the experiment and show run details with the widget.

hyperdrive_run = exp.submit(hyperdrive_config)

WARNING - If 'script' has been provided here and a script file name has been specified in 'run_config', 'script' provided in ScriptRunConfig initialization will take precedence.


In [23]:
import joblib
# Get your best run and save the model from that run.

print(hyperdrive_run.get_status())

if hyperdrive_run.get_status() == "Completed":
    best_run = hyperdrive_run.get_best_run_by_primary_metric()
    best_run_metrics = best_run.get_metrics()
    print("Best Run ", best_run.id, "= " ,best_run_metrics['Accuracy'])


Completed
Best Run  HD_dfa72971-ebd2-4b6a-ad03-c96b6b8f560a_92 =  0.9180576631259484


In [24]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

ds = TabularDatasetFactory.from_delimited_files("https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv")

In [25]:
from train import clean_data

# Use the clean_data function to clean your data.
x, y = clean_data(ds)

In [26]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    task="classification",
    primary_metric="accuracy",
    training_data=ds,
    label_column_name="y",
    n_cross_validations=3)

In [28]:
# Submit your automl run

automl_run = exp.submit(automl_config)

WARNING - Running on local machine. Note that local runs always run synchronously even if you use the parameter 'show_output=False'


In [ ]:
# Retrieve and save your best automl model.

print(automl_run.get_status())

if automl_run.get_status() == "Completed":
    best_run = automl_run.get_best_run_by_primary_metric()
    best_run_metrics = best_run.get_metrics()
    print("Best Run " +best_run.id+ "= " +best_run_metrics['Accuracy'])